In [9]:
import librosa, torch, pathlib, numpy as np
from tqdm import tqdm

# percorso ai .wav originali
ROOT = pathlib.Path('../data/raw/genres')   #  .. = sali di una cartella
# cartella di destinazione per i tensori .pt
OUT  = pathlib.Path('../data/processed')
OUT.mkdir(parents=True, exist_ok=True)

print("WAV da convertire:", len(list(ROOT.rglob('*.wav'))))

WAV da convertire: 1000


In [10]:
# Seconda cella – conversione WAV → Mel tensor (.pt) con skip dei file corrotti

bad_files = []   # terrà i nomi dei WAV che non si riescono a leggere

for wav in tqdm(list(ROOT.rglob("*.wav")), desc="Mel-spectrograms"):
    # 1) carica audio mono a 22 050 Hz
    try:
        y, _ = librosa.load(str(wav), sr=22_050, mono=True)
    except Exception as e:
        print(f"⚠️  Salto file corrotto: {wav.name}  ({e})")
        bad_files.append(wav.name)
        continue                           # passa al file successivo

    # 2) spettrogramma Mel 128 × T
    spec = librosa.feature.melspectrogram(
        y=y, sr=22_050,
        n_fft=2048, hop_length=512,
        n_mels=128, fmin=20, fmax=11_000
    )

    # 3) scala log (dB)
    spec_db = librosa.power_to_db(spec, ref=np.max)

    # 4) tensor 1 × 128 × 128 (pad o crop se necessario)
    if spec_db.shape[1] < 128:                              # clip troppo corta
        spec_db = np.pad(spec_db, ((0, 0), (0, 128 - spec_db.shape[1])),
                         mode="constant")
    spec_db = spec_db[:, :128]                              # clip troppo lunga
    tensor = torch.tensor(spec_db, dtype=torch.float32).unsqueeze(0)

    # 5) salva
    torch.save(tensor, OUT / f"{wav.stem}.pt")

print("\n--- Pre-processing completato ---")
print("File corrotti saltati:", len(bad_files))
if bad_files:
    print("Elenco:", bad_files)


Mel-spectrograms:  94%|█████████▍| 942/1000 [00:11<00:00, 79.62it/s]/var/folders/kf/8f7ds_x17c9b53gd9dq5n8pc0000gn/T/ipykernel_9367/3085427474.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, _ = librosa.load(str(wav), sr=22_050, mono=True)
Mel-spectrograms:  96%|█████████▌| 960/1000 [00:12<00:00, 80.23it/s]

⚠️  Salto file corrotto: jazz.00054.wav  ()


Mel-spectrograms: 100%|██████████| 1000/1000 [00:12<00:00, 78.84it/s]


--- Pre-processing completato ---
File corrotti saltati: 1
Elenco: ['jazz.00054.wav']
